In [35]:
from datetime import datetime
import pandas as pd

In [9]:
dat = pd.read_csv('../data/raw/case_dataset.csv')

In [10]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999999 entries, 0 to 2999998
Data columns (total 18 columns):
Unnamed: 0          int64
row_id              int64
is_clicked          int64
timestamp           int64
channel             int64
site_id             object
site_category       object
app_id              object
app_category        object
device_id           object
device_ip           object
device_model        object
device_type         int64
device_conn_type    int64
P1                  int64
P2                  int64
P3                  int64
P4                  int64
dtypes: int64(11), object(7)
memory usage: 412.0+ MB


In [12]:
# drop redundant index col and use row_id as index
dat = dat.drop(['Unnamed: 0'], axis=1)
dat = dat.set_index('row_id')
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2999999 entries, 1 to 2999999
Data columns (total 16 columns):
is_clicked          int64
timestamp           int64
channel             int64
site_id             object
site_category       object
app_id              object
app_category        object
device_id           object
device_ip           object
device_model        object
device_type         int64
device_conn_type    int64
P1                  int64
P2                  int64
P3                  int64
P4                  int64
dtypes: int64(9), object(7)
memory usage: 389.1+ MB


In [26]:
# check target label distribution
dat['is_clicked'].value_counts()

0    2486391
1     513608
Name: is_clicked, dtype: int64

In [22]:
dat['channel'].value_counts()

0    2233305
1     762286
5       2615
2       1058
7        428
4        298
3          9
Name: channel, dtype: int64

Convert some features to categorical. 
N.B. Not sure about channel. The description mentions 'position' which suggests an ordinal quality, but both the name 'channel' and the distribution of the value suggest a categorical nature. Normally I'd validate but for now I'll go for categorical.

In [20]:
categorical_features = [c for c in dat.columns if c not in ['is_clicked', 'timestamp']]
dat = dat.astype({c: 'category' for c in categorical_features})
dat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2999999 entries, 1 to 2999999
Data columns (total 16 columns):
is_clicked          int64
timestamp           int64
channel             int64
site_id             category
site_category       category
app_id              category
app_category        category
device_id           category
device_ip           category
device_model        category
device_type         category
device_conn_type    category
P1                  category
P2                  category
P3                  category
P4                  category
dtypes: category(13), int64(3)
memory usage: 216.0 MB


In [25]:
cols = [c for c in dat.columns if c not in ['is_clicked', 'timestamp']]
for c in cols:
    print(f'Feature {c} has {dat[c].nunique()} distinct values')

Feature channel has 7 distinct values
Feature site_id has 2716 distinct values
Feature site_category has 22 distinct values
Feature app_id has 3647 distinct values
Feature app_category has 30 distinct values
Feature device_id has 252147 distinct values
Feature device_ip has 819835 distinct values
Feature device_model has 5770 distinct values
Feature device_type has 4 distinct values
Feature device_conn_type has 4 distinct values
Feature P1 has 9 distinct values
Feature P2 has 188 distinct values
Feature P3 has 4 distinct values
Feature P4 has 44 distinct values


Features with many distinct values cannot be sensibly used, so we'll only keep channel, site_category, app_category, device_type, device_conn_type

In [27]:
dat['site_category'].value_counts()

50e219e0    1150046
f028772b     876625
28905ebd     659034
3e814130     217319
f66779e6      31043
335d28a8      16620
76b2941d      12260
75fa27f6      11184
0569f928       7892
72722551       7046
70fb0e29       5251
c0dd3be3       2420
a818d37a       1372
dedf689d       1281
e787de0e        339
42a36e14         91
bcf865d9         61
5378d028         58
8fd0aea4         47
9ccfa2ea          7
74073276          2
110ab22d          1
Name: site_category, dtype: int64

In [28]:
dat['app_category'].value_counts()

07d7df22    1998238
0f2161f8     662884
cef3e649     112797
f95efa07     101468
8ded1f7a      88521
d1327cf5      16597
09481d60       6794
75d80bbe       3434
4ce2e9fc       2314
fc6fa53d       2127
dc97ec06       1220
879c24eb       1053
a3c42688        842
0f9a328c        532
2281a340        312
4681bb9d        256
a86a3e89        199
79f0b860        141
0bfbc358        112
8df2e842         83
7113d72a         26
a7fd01ec         21
18b1e0be         16
4b7ade46          3
5326cf99          3
86c1a5a3          2
2fc4f2aa          1
52de74cf          1
ef03ae90          1
71af18ce          1
Name: app_category, dtype: int64

In [29]:
dat['device_type'].value_counts()

1    2815342
0     137429
4      42321
5       4907
Name: device_type, dtype: int64

In [30]:
dat['device_conn_type'].value_counts()

0    2687845
2     247312
3      60738
5       4104
Name: device_conn_type, dtype: int64

In [33]:
# check missing values
# wow no missing values!
dat.isnull().sum(axis=0)

is_clicked          0
timestamp           0
channel             0
site_id             0
site_category       0
app_id              0
app_category        0
device_id           0
device_ip           0
device_model        0
device_type         0
device_conn_type    0
P1                  0
P2                  0
P3                  0
P4                  0
dtype: int64

In [36]:
# parse datetime from timestamp and create datetime features
dat['datetime'] = dat['timestamp'].apply(lambda x: datetime.strptime(str(x), '%y%m%d%H'))
dat = dat.assign(
    year = dat['datetime'].dt.year,
    month = dat['datetime'].dt.month,
    month_day = dat['datetime'].dt.day,
    is_month_start = dat['datetime'].dt.is_month_start,
    is_month_end = dat['datetime'].dt.is_month_end,
    week_day = dat['datetime'].dt.weekday,
    is_weekend = dat['datetime'].dt.weekday >= 5,
    hour = dat['datetime'].dt.hour,
    is_working_hour = ((dat['datetime'].dt.hour >= 8) & (dat['datetime'].dt.hour <= 18)),
    is_night_hour = ((dat['datetime'].dt.hour >= 23) & (dat['datetime'].dt.hour <= 5))
)

In [37]:
print(dat['year'].value_counts())
print(dat['month'].value_counts())
print(dat['month_day'].value_counts())

2014    2999999
Name: year, dtype: int64
10    2999999
Name: month, dtype: int64
21    2999999
Name: month_day, dtype: int64


All data is from one single day! Not only does this mean we can't use most of these datetime features, but it's a bad idea to build a model on only this data, since all date/time-related influences will be unaccounted for.